In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

import sys
import os
sys.path.append(os.path.abspath(os.path.join('../scripts')))

#Import modules for feature selection
from featSelectFunctions.GeneticAlgoVarSelectCV3 import GeneticAlgoVarSelectCV
from KfoldLinearModels import KfoldLinearRegression
from featSelectFunctions.LassoVarSelectCV import LassoVarSelectCV
from featSelectFunctions.MRMR import mRMR_VarSelect
from featSelectFunctions.RFE_SVMCV import RFE_SVM_v2
from featSelectFunctions.EnetVarSelectCV import EnetVarSelectCV
from featSelectFunctions.fwrdFeatureSelection import FwrdFeatureSelection



import os


In [3]:
def rrmse(obs, Pred):
    return np.sqrt(np.mean(((obs - Pred)/obs)**2))

def rmse(obs, Pred):
    return np.sqrt(np.mean(((obs - Pred))**2))

factkm = (1000)/(60*60)

In [4]:
Root_data_path = os.path.abspath(os.path.join('../Data'))
data = pd.read_csv(os.path.join(Root_data_path ,"data_train.csv"))

Response = ['1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14']


In [5]:
Explanatory = np.array(['EPSG3979_x', 'EPSG3979_y', 'ASP100m.tif', 'ASP500m.tif',
       'ASP1000m.tif', 'ASP1500m.tif', 'ASP2000m.tif', 
       'MXC100m.tif', 'MXC500m.tif', 'MXC1000m.tif', 'MXC1500m.tif',
       'MXC2000m.tif', 'MNC100m.tif', 'MNC500m.tif', 'MNC1000m.tif',
       'MNC1500m.tif', 'MNC2000m.tif', 'RGI100m.tif', 'RGI500m.tif',
       'RGI1000m.tif', 'RGI1500m.tif', 'RGI2000m.tif', 'SLP100m.tif',
       'SLP500m.tif', 'SLP1000m.tif', 'SLP1500m.tif', 'SLP2000m.tif',
       'DVE100m.tif', 'DVE500m.tif', 'DVE1000m.tif',
       'DVE1500m.tif', 'DVE2000m.tif', 'DFE100m.tif', 'DFE500m.tif',
       'DFE1000m.tif', 'DFE1500m.tif', 'DFE2000m.tif', 'EPC100m.tif',
       'EPC500m.tif', 'EPC1000m.tif', 'EPC1500m.tif', 'EPC2000m.tif',
       'RTP100m.tif', 'RTP500m.tif', 'RTP1000m.tif', 'RTP1500m.tif',
       'RTP2000m.tif','DoG_1_5.tif', 'DoG_1_10.tif', 'DoG_5_10.tif', 'DoG_3_5.tif',
       'DoG_10_20.tif', 'DoG_10_15.tif', 'DoG_1_20.tif', 'DoG_5_20.tif',
       'LoG_1m.tif', 'LoG_5m.tif', 'LoG_10m.tif', 'LoG_15m.tif',
       'LoG_20m.tif', 'GCV100m.tif', 'GCV500m.tif', 'GCV1000m.tif',
       'GCV1500m.tif', 'GCV2000m.tif', 'ALOS_100m.tif',
       'MINC100m.tif', 'MINC500m.tif', 'MINC1000m.tif', 'MINC1500m.tif',
       'MINC2000m.tif', 'PCUR1000m.tif', 'PCUR1500m.tif', 'PCUR2000m.tif',
       'PCUR100m.tif', 'PCUR500m.tif', 'TCUR100m.tif', 'TCUR500m.tif',
       'TCUR1000m.tif', 'TCUR1500m.tif', 'TCUR2000m.tif', 'SDS100m.tif',
       'SDS500m.tif', 'SDS1000m.tif', 'SDS1500m.tif', 'SDS2000m.tif',
       'SAR100m.tif', 'SAR500m.tif', 'SAR1000m.tif', 'SAR1500m.tif',
       'SAR2000m.tif', 'TOCUR100m.tif', 'TOCUR500m.tif', 'TOCUR1000m.tif',
       'TOCUR1500m.tif', 'TOCUR2000m.tif', 'RoughL_100m', 'RoughL_1km',
       'RoughL_1km5',  'RoughL_2km', 
       'RoughL_500m',  'SeaDist', 'GMP_1', 'GMP_2', 'GMP_3',
       'GMP_5', 'GMP_6', 'GMP_7', 'GMP_8', 'GMP_9', 'GMP_10', 
       'PLC_5', 'PLC_6', 'PLC_7'])

# Enet 

In [6]:
#The parameters were initially selected see the notebook "Lasso Enet Parameters Search"
paraEnet = [(0.4111111111111111, 0.9333333333333333),
 (0.4111111111111111, 0.9333333333333333),
 (0.1, 1.0),
 (0.4111111111111111, 0.6000000000000001),
 (0.25555555555555554, 0.8),
 (0.33333333333333337, 0.6666666666666667),
 (0.25555555555555554, 0.8666666666666667),
 (0.6444444444444445, 0.6666666666666667),
 (0.48888888888888893, 0.7333333333333334),
 (0.5666666666666667, 0.6000000000000001),
 (0.8, 0.4666666666666667),
 (0.1, 0.9333333333333333),
 (0.1, 0.8666666666666667),
 (0.1, 0.7333333333333334),
 (0.6444444444444445, 0.4),
 (0.5666666666666667, 0.4),
 (0.4111111111111111, 0.5333333333333333)]

EnetFeat = []

for VarY, Para in zip(Response, paraEnet):
    EnetvarX, Para = EnetVarSelectCV(data, VarY, Explanatory, [Para[0]], [Para[1]])
    EnetFeat.append(EnetvarX)

Progress ... 1 / 1


# LASSO

In [8]:
LassoFeat = []

#The parameters were initially selected see the notebook "Lasso Enet Parameters Search"
Alphas = [0.35306122448979593,
 0.3857142857142858,
 0.4102040816326531,
 0.36122448979591837,
 0.35306122448979593,
 0.3204081632653062,
 0.3040816326530612,
 0.3122448979591837,
 0.29591836734693877,
 0.3204081632653062,
 0.40204081632653066,
 0.3285714285714286,
 0.3448979591836735,
 0.3204081632653062,
 0.29591836734693877,
 0.3857142857142858,
 0.41836734693877553]

for VarY, al in zip(Response, Alphas):
    LassovarX = LassoVarSelectCV(data, VarY, Explanatory, [al])
    LassoFeat.append(LassovarX)

Progress ...0 / 1


# Genetic Algo

In [ ]:
GAFeat = []
ngen = 100
for VarY in Response:
    BestSolution, _, _ = GeneticAlgoVarSelectCV(data, 50, Explanatory, VarY, ngen,  FitnessWratio=0.5)
    varx = BestSolution
    GAFeat.append(varx)
    

# Maximum Relevance Minimum Redundancy 

In [ ]:
MRFeat = []
Ks = np.arange(2, 20)

for varY in Response:
    score = np.zeros(Ks.shape[0])
    for ix, k in enumerate(Ks):
        print(ix)
        varX = mRMR_VarSelect(data, Explanatory, varY, k)
        score[ix] = KfoldLinearRegression(data, varY, varX)["rrmse"]
    Kbest = Ks[np.argmin(score)]
    
    varXBest = np.array(mRMR_VarSelect(data, Explanatory, varY, Kbest))
    MRFeat.append(varXBest)
MRFeat = [np.array(featLs) for featLs in MRFeat]

# Forward Feature Selection 

In [ ]:
FFS_Feat = []

for varY in Response:
    varx = FwrdFeatureSelection(data=data, varX=list(Explanatory), varY=varY)
    FFS_Feat.append(varx)

# Recursive Feature Elimination SVM 

In [ ]:
RFEFeat = []

for VarY in Response:
    _, RFEvarX, _, _= RFE_SVM_v2(data=data, VarY=VarY,
                                     VarX=Explanatory, 
                                     SVRParaSearchC=np.linspace(0.1, 10, 100))
    RFEFeat.append(RFEvarX)
    

# Saving Data all

In [ ]:
(pd.DataFrame( [list(feat) for feat in [LassoFeat, EnetFeat, GAFeat, MRFeat, RFEFeat, FFS_Feat]], 
             index=["LassoFeat", "EnetFeat", "GAFeat", "MRFeat", "RFEFeat", "FFSFeat"], 
            columns=["P"+str(i) for i in range(1, 18)])
 .to_csv(os.path.abspath(r"..\Results\varSelect.csv")))

In [ ]:
[LassoFeat, EnetFeat, GAFeat, MRFeat, RFEFeat]